<a href="https://colab.research.google.com/github/BenWilop/WSG_games/blob/main/playground_WSG_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import getpass
import dotenv
import wandb
import os
dotenv.load_dotenv(os.path.join('/homes/55/bwilop/wsg/private/', 'vscode-ssh.env'))
api_key = os.getenv('WANDB_API_KEY')
wandb.login(key=api_key)

data_folder = '/homes/55/bwilop/wsg/data/'
experiment_folder = '/homes/55/bwilop/wsg/experiments/'

: 

In [ ]:
%load_ext autoreload
%autoreload 2
import torch as t
import torch.nn as nn
from torch.nn.functional import cross_entropy, softmax
from torch import Tensor
from jaxtyping import Float
from tqdm import tqdm
from transformer_lens import HookedTransformerConfig, HookedTransformer
import json
from typing import Callable, Any
import einops
import time
import uuid
import glob
from copy import deepcopy
import matplotlib.pyplot as plt
from datetime import datetime
import torch.nn as nn

from wsg_games.tictactoe.evals import *
from wsg_games.tictactoe.data import *
from wsg_games.tictactoe.game import *

from wsg_games.tictactoe.analysis.analyse_data import *
from wsg_games.tictactoe.analysis.visualize_game import *

from wsg_games.tictactoe.train.create_models import *
from wsg_games.tictactoe.train.save_load_models import *
from wsg_games.tictactoe.train.train import *
from wsg_games.tictactoe.train.finetune import *
from wsg_games.tictactoe.train.pretrain import *

device = t.device("cuda" if t.cuda.is_available() else "cpu")
print(device)

# Data

In [ ]:
# tictactoe_data = cache_tictactoe_data(data_folder + 'tictactoe_data.pkl')
tictactoe_data = cache_tictactoe_data_random(data_folder + 'tictactoe_data_random_STRONG_RULE_REVERSE_RULE.pkl')
# tictactoe_data = cache_tictactoe_data_random(data_folder + 'tictactoe_data_random_STRONG_RULE_NO_DIAGONAL.pkl')
print_data_statistics(tictactoe_data)

In [ ]:
# tictactoe_train_data, weak_finetune_data, tictactoe_val_data, tictactoe_test_data = train_test_split_tictactoe(tictactoe_data, 0.5, 0.25, 0.05, 0.20, device, 1234)
tictactoe_train_data, tictactoe_weak_finetune_data, tictactoe_val_data, tictactoe_test_data = train_test_split_tictactoe_first_two_moves_no_overlap(tictactoe_data, 42, 15, 5, 10, device, 1234)

print("Shape Train: ", tictactoe_train_data.games_data.shape)
print("Shape Weak Finetune: ", tictactoe_weak_finetune_data.games_data.shape)
print("Shape Validation: ", tictactoe_val_data.games_data.shape)
print("Shape Test:  ", tictactoe_test_data.games_data.shape)

In [ ]:
# leakage_pct = calculate_leakage_percentage(tictactoe_train_data, tictactoe_test_data)
# print(f"Leakage Percentage: {leakage_pct:.2f}%")

In [ ]:
# plot_train_test_prefix_histograms(tictactoe_train_data, tictactoe_test_data)

In [ ]:
tictactoe_train_data = create_hard_label_tictactoe_data(tictactoe_train_data, num_samples=1)
tictactoe_weak_finetune_data = create_hard_label_tictactoe_data(tictactoe_weak_finetune_data, num_samples=1)
tictactoe_val_data = create_hard_label_tictactoe_data(tictactoe_val_data, num_samples=1)

In [ ]:
print("Shape Train: ", tictactoe_train_data.games_data.shape)
tictactoe_train_data.weak_goals_labels[0][:3]

In [ ]:
print("Shape Weak Finetune: ", tictactoe_weak_finetune_data.games_data.shape)
tictactoe_weak_finetune_data.weak_goals_labels[0][:3]

In [ ]:
print("Shape Val: ", tictactoe_val_data.games_data.shape)
tictactoe_val_data.weak_goals_labels[0][:3]

In [ ]:
print("Shape Test:  ", tictactoe_test_data.games_data.shape)
tictactoe_test_data.weak_goals_labels[0][:3]

# Train model

In [ ]:
# raise Exception()

### Parameters

In [ ]:
training_cfg = get_training_cfg()
model_sizes = get_model_sizes()
print_model_ratios(model_sizes, get_model_config)

### wandb

### Train loop

In [ ]:
from wsg_games.tictactoe.evals import *
from wsg_games.tictactoe.data import *
from wsg_games.tictactoe.game import *

from wsg_games.tictactoe.analysis.analyse_data import *
from wsg_games.tictactoe.analysis.visualize_game import *

from wsg_games.tictactoe.train.create_models import *
from wsg_games.tictactoe.train.save_load_models import *
from wsg_games.tictactoe.train.train import *
from wsg_games.tictactoe.train.finetune import *
from wsg_games.tictactoe.train.pretrain import *

In [ ]:
# project_name = "tictactoe_playground"
# model_size = "large"
# model_cfg = get_model_config(model_size)
# wandb.finish()
# model, _, _ = run_full_training(project_name, model_size, Goal.STRONG_GOAL, tictactoe_train_data, tictactoe_val_data, tictactoe_test_data, training_cfg, model_cfg)

### Experiments

In [ ]:
project_name = "tictactoe_pretrained_reverse_rule_no_overlap_split_start_third_200k"
# project_name = "tictactoe_pretrained_forbidden_diagonals_no_overlap_split_start_third_200k"
# pretrain_models(experiment_folder, project_name, tictactoe_train_data, tictactoe_val_data, tictactoe_test_data, training_cfg, get_model_config)

In [ ]:
# plot_loss_pretrain_models(experiment_folder, project_name, tictactoe_test_data)

In [ ]:
# project_name = "tictactoe_pretrained_reverse_rule_no_overlap_split_start_third_200k"
# weak_model_weak_goals = load_model(project_name, "mini", Goal.WEAK_GOAL, experiment_folder)
# strong_model_weak_goals = load_model(project_name, "medium", Goal.WEAK_GOAL, experiment_folder)
# strong_model_strong_goals = load_model(project_name, "medium", Goal.STRONG_GOAL, experiment_folder)
# quick_evaluation("Weak model weak goal", weak_model_weak_goals, tictactoe_test_data)
# quick_evaluation("Strong model weak goal", strong_model_weak_goals, tictactoe_test_data)
# quick_evaluation("Strong model strong goal", strong_model_strong_goals, tictactoe_test_data)
# strong_model_finetuned = deepcopy(strong_model_strong_goals)

# project_name = "tictactoe_playground"
# timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M")
# experiment_name = f"experiment_finetuning_{timestamp}"
# # Use validation data as weak_train_data
# finetune_strong_with_weak(project_name, experiment_name, weak_model_weak_goals, strong_model_finetuned, tictactoe_weak_finetune_data, tictactoe_test_data, adapted_training_cfg)

In [ ]:
pretrained_project_name = "tictactoe_pretrained_reverse_rule_no_overlap_split_start_third_200k" 
finetuned_project_name = "finetune_sweep_test"
# finetune_sweep(pretrained_project_name, finetuned_project_name, experiment_folder, tictactoe_weak_finetune_data, tictactoe_val_data, tictactoe_test_data, training_cfg)

In [ ]:
import os
import glob
import torch as t

# Helper: compute loss on test data
def get_weak_loss(model, test_data: TicTacToeData, n_samples: int = 20000) -> float:
    model.eval()
    with t.no_grad():
        sample = random_sample_tictactoe_data(test_data, n_samples)
        logits = model(sample.games_data)
        loss = cross_entropy(rearrange(logits), rearrange(sample.weak_goals_labels)).item()
    return loss


def visualize_finetuning_results(pretrained_project_name: str, finetuned_project_name: str, 
                                 experiment_folder: str, test_data: TicTacToeData, 
                                 model_sizes: list):
    """
    For each weak_size and for every strong_size larger than weak_size,
    compute the recovered percentage:
    
        recovered % = (L_weak - L_model) / (L_weak - L_strong_baseline) * 100
    
    where:
      - L_weak: loss of the weak model (evaluated on weak_goals)
      - L_strong_baseline: loss of the pretrained strong model on the weak goal
      - L_model: loss of either the pretrained strong model (for the left plot) or the finetuned model (for the right plot)
      
    The x-axis is the number of parameters in the strong model (log-scaled).
    One line is drawn per weak model size, connecting datapoints for each strong size.
    Two subplots are created: left for pretrained models and right for finetuned models.
    """
    data_pretrained = {}  # { weak_size: list of (num_params, recovered %) }
    data_finetuned = {}
    
    for weak_size in model_sizes:
        weak_model = load_model(pretrained_project_name, weak_size, 
                                Goal.WEAK_GOAL, experiment_folder)
        if not weak_model:
            print(f"Weak model of size {weak_size} not found, skipping.")
            continue
        
        # Get loss of weak model (using weak labels)
        L_weak = get_weak_loss(weak_model, test_data)
        data_pretrained[weak_size] = []
        data_finetuned[weak_size] = []
        
        for strong_size in model_sizes:
            if model_sizes.index(strong_size) <= model_sizes.index(weak_size):
                continue
            
            strong_model_weak_goal = load_model(pretrained_project_name, strong_size, 
                                                 Goal.WEAK_GOAL, experiment_folder)
            strong_model_strong_goal = load_model(pretrained_project_name, strong_size, 
                                                 Goal.STRONG_GOAL, experiment_folder)
            if not strong_model_weak_goal or not strong_model_strong_goal:
                print(f"Strong models of size {strong_size} not found, skipping.")
                continue
            
            num_params = count_parameters(strong_model_weak_goal)
            L_strong_baseline = get_weak_loss(strong_model_weak_goal, test_data)
            L_strong_pretrained_on_weak_goal = get_weak_loss(strong_model_strong_goal, test_data)
            
            # Left plot
            recovered_pretrained = (L_weak - L_strong_pretrained_on_weak_goal) / (L_weak - L_strong_baseline) * 100 

            # Right plot
            finetuned_model = load_finetuned_model(finetuned_project_name, weak_size, strong_size, experiment_folder)
            if not finetuned_model:
                print(f"Finetuned model of size {strong_size} not found, skipping.")
                continue

            L_finetuned = get_weak_loss(finetuned_model, test_data)
            recovered_finetuned = (L_weak - L_finetuned) / (L_weak - L_strong_baseline) * 100
            
            data_pretrained[weak_size].append((num_params, recovered_pretrained))
            data_finetuned[weak_size].append((num_params, recovered_finetuned))
    
    # Create two subplots: left for pretrained, right for finetuned.
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot pretrained data.
    for weak_size, points in data_pretrained.items():
        if points:
            points.sort(key=lambda x: x[0])
            x_vals = [p[0] for p in points]
            y_vals = [p[1] for p in points]
            axes[0].plot(x_vals, y_vals, marker='o', label=f"Weak {weak_size}")
    axes[0].set_xscale('log')
    axes[0].set_xlabel("Number of Parameters (log scale)")
    axes[0].set_ylabel("Recovered %")
    axes[0].set_title("Pretrained Models")
    axes[0].legend()
    axes[0].grid(True, which="both", ls="--")
    
    # Plot finetuned data.
    for weak_size, points in data_finetuned.items():
        if points:
            points.sort(key=lambda x: x[0])
            x_vals = [p[0] for p in points]
            y_vals = [p[1] for p in points]
            axes[1].plot(x_vals, y_vals, marker='o', label=f"Weak {weak_size}")
    axes[1].set_xscale('log')
    axes[1].set_xlabel("Number of Parameters (log scale)")
    axes[1].set_ylabel("Recovered %")
    axes[1].set_title("Finetuned Models")
    axes[1].legend()
    axes[1].grid(True, which="both", ls="--")
    
    plt.tight_layout()
    plt.show()

visualize_finetuning_results(pretrained_project_name, finetuned_project_name, 
                                 experiment_folder, tictactoe_test_data, 
                                 ["nano", "micro", "mini", "small", "medium", "large", "huge"])

In [ ]:
raise Exception()

# Inspect model

In [ ]:
project_name = "tictactoe_pretrained_reverse_rule_no_overlap_split_start_third_200k"
model = load_model(project_name, "large", Goal.STRONG_GOAL, experiment_folder)
quick_evaluation("model weak goal", model, tictactoe_test_data)

In [ ]:
W = model.embed.W_E
W_normed = W / W.norm(dim=1, keepdim=True)

# Compute cosine similarity matrix and move to CPU as a NumPy array
cosine_sim = (W_normed @ W_normed.T).detach().cpu().numpy()

im = plt.imshow(cosine_sim)
plt.title("Cosine similarities of each pair of 2D feature embeddings")
plt.gcf().set_size_inches(6, 6)

# Add a colorbar with a label
cbar = plt.colorbar(im)
cbar.set_label("Cosine similarity value")

plt.show()

In [ ]:
W.shape

In [ ]:
train_sample = random_sample_tictactoe_data(tictactoe_train_data, 1)
train_logits = model(train_sample.games_data)
soft_labels = softmax(train_logits, dim=-1)
hard_labels = F.one_hot(train_sample.weak_goals_labels.argmax(dim=-1), num_classes=train_logits.shape[-1]).float()

print(train_sample.weak_goals_labels.shape)
print(train_logits.shape)
print(soft_labels.shape)
print(hard_labels.shape)

In [ ]:
train_sample.weak_goals_labels

In [ ]:
soft_labels.clip(0.01)

In [ ]:
hard_labels

In [ ]:
board = Board()
print(get_best_moves(board, Goal.WEAK_GOAL))
board.make_move(1)
print(get_best_moves(board, Goal.WEAK_GOAL))

In [ ]:
tictactoe_test_data.games_data[600]

In [ ]:
tictactoe_test_data.weak_goals_labels[2]

In [ ]:
tictactoe_test_data.games_data[25][:]

In [ ]:
softmax(model(tictactoe_test_data.games_data[25][:]), dim=-1).clip(0.01)

In [ ]:
skip_to = 0
train_dataset = TensorDataset(
    tictactoe_train_data.games_data,
    tictactoe_train_data.random_move_labels,
    tictactoe_train_data.weak_goals_labels,
    tictactoe_train_data.strong_goals_labels,
)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
i = 0
for games, random_labels, weak_labels, strong_labels in train_loader:
    logits = model(games)
    probs = torch.softmax(logits, dim=-1)
    illegal_mask = (random_labels == 0).float()
    illegal_move_probs = (probs * illegal_mask).sum(dim=-1)
    avg_illegal = illegal_move_probs.mean().item()

    print(i, avg_illegal)

    i += 1
    if i > 100:
        break

In [ ]:
visualize_game(tictactoe_test_data, game_id=0, model=model)

# Sample games

In [ ]:
samples = sample_games(model, 1, 100)

In [ ]:
evals.eval_model(samples)

In [ ]:
sample = samples[1]
print(sample)

In [ ]:
from alphatoe.game import Board, State

def check(game: list[int]) -> bool:
    board = Board()
    for move in game[1:-1]:
        print(move)
        if board.game_state == State.ONGOING:
            try:
                board.make_move(move)
            except:
                return True
        elif move == 9:
            pass
        else:
            return True
    return False

check(sample)

In [ ]:
game.play_game(sample)
print("\nPLAY:")
board = game.Board()
for move in sample:
  if move not in [9, 10]:
    board.make_move(move)
    board.draw_board()
    print(".")

In [ ]:
print(evals.model_vs_minimax(model, True), "\n")
print(evals._check_minimax_win_rate(model, 100))

In [ ]:
evals.get_error_rate(samples)